In [43]:
import pandas as pd
import ast
import os
from pymongo import MongoClient
from pymongo.errors import BulkWriteError
import ipywidgets as widgets
from IPython.display import display, clear_output
import re
from itertools import combinations
from collections import defaultdict, Counter
import uuid
from datetime import datetime

# Conexión MongoDB
client = MongoClient(os.environ["ATLASMONGODB_CONNECTION_STRING"])
db = client["fraud_db"]
collection = db["transactions"]
productos_col = db["products"]

def clean_widget_callbacks(widget):
    """Limpia todos los callbacks de un widget"""
    if hasattr(widget, '_click_handlers'):
        widget._click_handlers.callbacks.clear()
    if hasattr(widget, '_observe_callbacks'):
        widget._observe_callbacks.clear()

# ======================== TAB A: EXTRACCIÓN DE DATOS ========================

# Widgets para carga de datos
file_picker = widgets.FileUpload(accept='.xlsx', multiple=False)
ruta_textbox = widgets.Text(value='', description='Archivo:', disabled=True)
btn_subir = widgets.Button(description="Subir a MongoDB", button_style='success')
progress = widgets.IntProgress(value=0, min=0, max=100, description='Progreso:')
btn_generar_maestro = widgets.Button(description="Generar Maestro", button_style='info')
btn_resumen = widgets.Button(description="Resumen Básico", button_style='warning')
output_carga = widgets.Output()

def on_file_selected(change):
    with output_carga:
        if file_picker.value:
            nombre_archivo = file_picker.value[0]['name']
            ruta_textbox.value = nombre_archivo

def on_subir_clicked(b):
    with output_carga:
        output_carga.clear_output()
        if not file_picker.value:
            print("⚠️ Por favor selecciona un archivo .xlsx.")
            return

        uploaded_file = file_picker.value[0]
        contenido = uploaded_file['content']
        with open("temp.xlsx", "wb") as f:
            f.write(contenido)

        df = pd.read_excel("temp.xlsx", engine="openpyxl")
        df['ITEM'] = df['ITEM'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
        df = df.rename(columns={'ID_Transaccion': '_id', 'ITEM': 'items'})
        registros = df.to_dict(orient="records")

        batch_size = 100
        total = len(registros)
        progress.max = total
        progress.value = 0
        insertados = 0

        for i in range(0, total, batch_size):
            batch = registros[i:i+batch_size]
            try:
                res = collection.insert_many(batch, ordered=False)
                insertados += len(res.inserted_ids)
            except BulkWriteError as bwe:
                errores = bwe.details.get("writeErrors", [])
                insertados += len(batch) - len(errores)
            progress.value = min(i + batch_size, total)

        print(f"✅ Procesados: {total} registros")
        print(f"📥 Insertados exitosamente: {insertados} registros")

def on_generar_maestro(b):
    with output_carga:
        output_carga.clear_output()
        print("🔄 Generando maestro de productos...")
        
        productos_col.delete_many({})
        print("🗑️ Colección 'products' limpiada")
        
        transacciones = list(collection.find({}, {"_id": 0, "items": 1}))
        items_flat = [item.strip() for t in transacciones for item in t['items']]
        unique_items = sorted(set(items_flat))

        productos = []
        for item in unique_items:
            match = re.match(r"(.+?)_(\d+)$", item.strip())
            if match:
                nombre, codigo = match.groups()
                categoria = nombre.strip()
            else:
                nombre = item.strip()
                codigo = "NA"
                categoria = "otros"
            productos.append({
                "codigo": codigo,
                "nombre": item.strip(),
                "categoria": categoria
            })

        productos_col.insert_many(productos)
        print(f"🎉 Maestro generado con {len(productos)} productos distintos.")
        
        df_prod = pd.DataFrame(productos)
        resumen = df_prod.groupby("categoria")["nombre"].count().reset_index()
        resumen.columns = ['Categoria', 'Productos_unicos']
        print("\n📊 Resumen por categoría:")
        display(resumen)

def on_resumen_clicked(b):
    with output_carga:
        output_carga.clear_output()
        print("📈 Generando resumen básico...")
        
        total_transacciones = collection.count_documents({})
        transacciones = list(collection.find({}, {"_id": 0, "items": 1}))
        items_flat = [item.strip() for t in transacciones for item in t['items']]
        
        print(f"📊 Total de transacciones: {total_transacciones}")
        print(f"🛒 Total de productos vendidos: {len(items_flat)}")
        print(f"🏷️ Productos únicos: {len(set(items_flat))}")
        
        # Top 10 productos
        top_productos = pd.Series(items_flat).value_counts().head(10)
        print("\n🏆 Top 10 productos más vendidos:")
        display(top_productos.reset_index().rename(columns={'index': 'Producto', 0: 'Cantidad'}))

# Asignar callbacks
clean_widget_callbacks(file_picker)
file_picker.observe(on_file_selected, names='value')
clean_widget_callbacks(btn_subir)
btn_subir.on_click(on_subir_clicked)
clean_widget_callbacks(btn_generar_maestro)
btn_generar_maestro.on_click(on_generar_maestro)
clean_widget_callbacks(btn_resumen)
btn_resumen.on_click(on_resumen_clicked)

# ======================== TAB B: ALGORITMO APRIORI ========================

# Widgets para Apriori - SEPARADO EN DOS ETAPAS
# Etapa 1: Itemsets frecuentes
minsup_slider = widgets.FloatSlider(
    value=0.05,  # 5% por defecto
    min=0.01,    # 1% mínimo
    max=0.30,    # 30% máximo
    step=0.01,   # Pasos de 1%
    description='Min Support:',
    style={'description_width': 'initial'},
    readout_format='.0%'  # Mostrar como porcentaje
)
btn_calcular_itemsets = widgets.Button(description="1. Calcular Itemsets Frecuentes", button_style='primary')
output_itemsets = widgets.Output()

# Etapa 2: Reglas de asociación
minconf_slider = widgets.FloatSlider(
    value=0.50,  # 50% por defecto
    min=0.10,    # 10% mínimo
    max=1.0,     # 100% máximo
    step=0.05,   # Pasos de 5%
    description='Min Confidence:',
    style={'description_width': 'initial'},
    readout_format='.0%'  # Mostrar como porcentaje
)
btn_generar_reglas = widgets.Button(description="2. Generar Reglas de Asociación", button_style='success', disabled=True)
output_reglas = widgets.Output()

# Variable global para almacenar el objeto Apriori
apriori_obj = None

class AprioriAlgorithm:
    def __init__(self, transactions, min_support, min_confidence):
        self.transactions = transactions
        self.min_support = min_support
        self.min_confidence = min_confidence
        self.itemsets = {}
        self.rules = []
    
    def get_support(self, itemset):
        """Calcula el soporte de un itemset"""
        count = 0
        for transaction in self.transactions:
            if set(itemset).issubset(set(transaction)):
                count += 1
        return count / len(self.transactions)
    
    def get_frequent_1_itemsets(self):
        """Obtiene los 1-itemsets frecuentes"""
        items = set()
        for transaction in self.transactions:
            for item in transaction:
                items.add(item)
        
        frequent_1_itemsets = []
        for item in items:
            support = self.get_support([item])
            if support >= self.min_support:
                frequent_1_itemsets.append(([item], support))
        
        return frequent_1_itemsets
    
    def generate_candidates(self, prev_frequent_itemsets, k):
        """Genera candidatos de tamaño k"""
        candidates = []
        items = [itemset for itemset, _ in prev_frequent_itemsets]
        
        for i in range(len(items)):
            for j in range(i + 1, len(items)):
                # Unir dos itemsets de tamaño k-1
                union = sorted(set(items[i] + items[j]))
                if len(union) == k and union not in candidates:
                    candidates.append(union)
        
        return candidates
    
    def apriori(self):
        """Ejecuta el algoritmo Apriori"""
        # Obtener 1-itemsets frecuentes
        frequent_itemsets = self.get_frequent_1_itemsets()
        self.itemsets[1] = frequent_itemsets
        
        k = 2
        while self.itemsets[k-1]:
            # Generar candidatos
            candidates = self.generate_candidates(self.itemsets[k-1], k)
            
            # Filtrar candidatos frecuentes
            frequent_k_itemsets = []
            for candidate in candidates:
                support = self.get_support(candidate)
                if support >= self.min_support:
                    frequent_k_itemsets.append((candidate, support))
            
            if frequent_k_itemsets:
                self.itemsets[k] = frequent_k_itemsets
                k += 1
            else:
                break
    
    def generate_rules(self):
        """Genera reglas de asociación"""
        self.rules = []
        
        # Para cada itemset de tamaño >= 2
        for k in range(2, len(self.itemsets) + 1):
            if k not in self.itemsets:
                continue
                
            for itemset, support in self.itemsets[k]:
                # Generar todas las posibles divisiones del itemset
                for i in range(1, len(itemset)):
                    for antecedent in combinations(itemset, i):
                        antecedent = list(antecedent)
                        consequent = [item for item in itemset if item not in antecedent]
                        
                        # Calcular confianza
                        antecedent_support = self.get_support(antecedent)
                        if antecedent_support > 0:
                            confidence = support / antecedent_support
                            
                            if confidence >= self.min_confidence:
                                self.rules.append({
                                    'antecedent': antecedent,
                                    'consequent': consequent,
                                    'support': support,
                                    'confidence': confidence
                                })

def on_calcular_itemsets(b):
    with output_itemsets:
        output_itemsets.clear_output()
        print("🔄 ETAPA 1: Calculando Itemsets Frecuentes...")
        print(f"📊 Parámetro: min_support = {minsup_slider.value:.1%}")
        print("=" * 50)
        
        # Obtener transacciones
        transacciones_db = list(collection.find({}, {"_id": 0, "items": 1}))
        transacciones = [t['items'] for t in transacciones_db]
        
        if not transacciones:
            print("❌ No hay transacciones cargadas.")
            return
        
        print(f"📦 Total de transacciones: {len(transacciones)}")
        
        # Calcular items únicos
        all_items = set()
        for trans in transacciones:
            all_items.update(trans)
        print(f"🏷️ Total de productos únicos: {len(all_items)}")
        
        # Crear y ejecutar Apriori solo para itemsets
        global apriori_obj
        apriori_obj = AprioriAlgorithm(transacciones, minsup_slider.value, minconf_slider.value)
        apriori_obj.apriori()
        
        # Mostrar resumen de itemsets
        print("\n📊 RESUMEN DE ITEMSETS FRECUENTES:")
        print("-" * 40)
        
        total_itemsets = 0
        max_size = 0
        
        for k, itemsets in apriori_obj.itemsets.items():
            if itemsets:
                total_itemsets += len(itemsets)
                max_size = k
                print(f"• {k}-itemsets: {len(itemsets)} encontrados")
        
        if total_itemsets > 0:
            print(f"\n✅ Total de itemsets frecuentes: {total_itemsets}")
            print(f"📏 Tamaño máximo de itemset: {max_size}")
            
            # Crear contenido HTML para mostrar TODOS los itemsets en scroll
            html_content = """
            <div style='margin-top: 20px;'>
                <h4>📋 TODOS LOS ITEMSETS FRECUENTES</h4>
                <div style='background-color: #f5f5f5; padding: 10px; border-radius: 5px; 
                            max-height: 400px; overflow-y: auto; border: 1px solid #ddd;'>
            """
            
            # Mostrar 1-itemsets y 2-itemsets juntos
            if 1 in apriori_obj.itemsets and apriori_obj.itemsets[1]:
                html_content += "<h5>1-itemsets (productos individuales):</h5><ul style='margin: 5px 0;'>"
                sorted_1_itemsets = sorted(apriori_obj.itemsets[1], key=lambda x: x[1], reverse=True)
                for itemset, support in sorted_1_itemsets:
                    html_content += f"<li><b>{itemset[0]}</b> - Soporte: {support:.1%} ({int(support * len(transacciones))} trans.)</li>"
                html_content += "</ul>"
            
            if 2 in apriori_obj.itemsets and apriori_obj.itemsets[2]:
                html_content += "<h5>2-itemsets (pares de productos):</h5><ul style='margin: 5px 0;'>"
                sorted_2_itemsets = sorted(apriori_obj.itemsets[2], key=lambda x: x[1], reverse=True)
                for itemset, support in sorted_2_itemsets:
                    html_content += f"<li><b>{{{', '.join(itemset)}}}</b> - Soporte: {support:.1%} ({int(support * len(transacciones))} trans.)</li>"
                html_content += "</ul>"
            
            # Mostrar itemsets de tamaño 3 o más
            for k in range(3, max_size + 1):
                if k in apriori_obj.itemsets and apriori_obj.itemsets[k]:
                    html_content += f"<h5>{k}-itemsets:</h5><ul style='margin: 5px 0;'>"
                    sorted_k_itemsets = sorted(apriori_obj.itemsets[k], key=lambda x: x[1], reverse=True)
                    for itemset, support in sorted_k_itemsets:
                        html_content += f"<li><b>{{{', '.join(itemset)}}}</b> - Soporte: {support:.1%} ({int(support * len(transacciones))} trans.)</li>"
                    html_content += "</ul>"
            
            html_content += "</div></div>"
            
            # Mostrar el HTML
            display(widgets.HTML(html_content))
            
            # Resumen estadístico
            print("\n📈 ANÁLISIS ESTADÍSTICO:")
            print("-" * 40)
            
            # Calcular estadísticas por tamaño de itemset
            for k in range(1, max_size + 1):
                if k in apriori_obj.itemsets and apriori_obj.itemsets[k]:
                    soportes = [support for _, support in apriori_obj.itemsets[k]]
                    print(f"\n{k}-itemsets:")
                    print(f"  • Cantidad: {len(soportes)}")
                    print(f"  • Soporte promedio: {sum(soportes)/len(soportes):.1%}")
                    print(f"  • Soporte máximo: {max(soportes):.1%}")
                    print(f"  • Soporte mínimo: {min(soportes):.1%}")
            
            # Habilitar el botón de reglas
            btn_generar_reglas.disabled = False
            print("\n✅ Itemsets calculados. Ahora puedes generar las reglas de asociación.")
        else:
            print("\n⚠️ No se encontraron itemsets frecuentes con el soporte mínimo especificado.")
            print("💡 Intenta reducir el valor de soporte mínimo.")
            btn_generar_reglas.disabled = True

def on_generar_reglas(b):
    with output_reglas:
        output_reglas.clear_output()
        
        if not apriori_obj or not apriori_obj.itemsets:
            print("❌ Primero debes calcular los itemsets frecuentes.")
            return
        
        print("🔄 ETAPA 2: Generando Reglas de Asociación...")
        print(f"📊 Parámetro: min_confidence = {minconf_slider.value:.1%}")
        print("=" * 50)
        
        # Actualizar confianza mínima y generar reglas
        apriori_obj.min_confidence = minconf_slider.value
        apriori_obj.generate_rules()
        
        if apriori_obj.rules:
            # Ordenar reglas por confianza
            apriori_obj.rules.sort(key=lambda x: x['confidence'], reverse=True)
            
            print(f"\n🔗 REGLAS DE ASOCIACIÓN ENCONTRADAS: {len(apriori_obj.rules)}")
            print("-" * 40)
            
            # Estadísticas de las reglas
            confidences = [r['confidence'] for r in apriori_obj.rules]
            supports = [r['support'] for r in apriori_obj.rules]
            
            print(f"\n📈 ESTADÍSTICAS DE LAS REGLAS:")
            print(f"• Confianza promedio: {sum(confidences)/len(confidences):.1%}")
            print(f"• Confianza máxima: {max(confidences):.1%}")
            print(f"• Confianza mínima: {min(confidences):.1%}")
            print(f"• Soporte promedio: {sum(supports)/len(supports):.1%}")
            
            # Crear contenido HTML para mostrar TODAS las reglas en scroll
            html_content = """
            <div style='margin-top: 20px;'>
                <h4>📋 TODAS LAS REGLAS DE ASOCIACIÓN</h4>
                <div style='background-color: #f5f5f5; padding: 15px; border-radius: 5px; 
                            max-height: 400px; overflow-y: auto; border: 1px solid #ddd;'>
                <table style='width: 100%; border-collapse: collapse;'>
                    <thead style='position: sticky; top: 0; background-color: #e0e0e0;'>
                        <tr>
                            <th style='padding: 8px; text-align: left; border-bottom: 2px solid #999;'>#</th>
                            <th style='padding: 8px; text-align: left; border-bottom: 2px solid #999;'>Antecedente → Consecuente</th>
                            <th style='padding: 8px; text-align: center; border-bottom: 2px solid #999;'>Confianza</th>
                            <th style='padding: 8px; text-align: center; border-bottom: 2px solid #999;'>Soporte</th>
                            <th style='padding: 8px; text-align: center; border-bottom: 2px solid #999;'>Lift</th>
                        </tr>
                    </thead>
                    <tbody>
            """
            
            for i, rule in enumerate(apriori_obj.rules, 1):
                ant_str = ", ".join(rule['antecedent'])
                cons_str = ", ".join(rule['consequent'])
                
                # Calcular lift
                cons_support = apriori_obj.get_support(rule['consequent'])
                lift = rule['confidence'] / cons_support if cons_support > 0 else 0
                
                # Color de fondo según el lift
                bg_color = '#d4edda' if lift > 1.2 else '#f8f9fa' if lift > 1 else '#fff3cd'
                
                html_content += f"""
                    <tr style='background-color: {bg_color}; border-bottom: 1px solid #ddd;'>
                        <td style='padding: 8px;'>{i}</td>
                        <td style='padding: 8px;'><b>[{ant_str}]</b> → <b>[{cons_str}]</b></td>
                        <td style='padding: 8px; text-align: center;'>{rule['confidence']:.1%}</td>
                        <td style='padding: 8px; text-align: center;'>{rule['support']:.1%}</td>
                        <td style='padding: 8px; text-align: center;'>{lift:.2f}</td>
                    </tr>
                """
            
            html_content += """
                    </tbody>
                </table>
                </div>
                <div style='margin-top: 10px; font-size: 12px; color: #666;'>
                    <b>Leyenda de colores:</b> 
                    <span style='background-color: #d4edda; padding: 2px 5px;'>Lift > 1.2 (asociación fuerte)</span>
                    <span style='background-color: #f8f9fa; padding: 2px 5px;'>Lift > 1 (asociación positiva)</span>
                    <span style='background-color: #fff3cd; padding: 2px 5px;'>Lift ≤ 1 (asociación débil/negativa)</span>
                </div>
            </div>
            """
            
            # Mostrar el HTML
            display(widgets.HTML(html_content))
            
            # Análisis adicional: Top reglas por lift
            print("\n🏆 TOP 10 REGLAS POR LIFT:")
            print("-" * 40)
            
            # Calcular lift para todas las reglas y ordenar
            rules_with_lift = []
            for rule in apriori_obj.rules:
                cons_support = apriori_obj.get_support(rule['consequent'])
                if cons_support > 0:
                    lift = rule['confidence'] / cons_support
                    rules_with_lift.append((rule, lift))
            
            rules_with_lift.sort(key=lambda x: x[1], reverse=True)
            
            for i, (rule, lift) in enumerate(rules_with_lift[:10], 1):
                ant_str = ", ".join(rule['antecedent'])
                cons_str = ", ".join(rule['consequent'])
                print(f"{i}. [{ant_str}] → [{cons_str}]")
                print(f"   Lift: {lift:.2f}, Confianza: {rule['confidence']:.1%}, Soporte: {rule['support']:.1%}")
            
            # Guardar reglas globalmente
            global apriori_rules
            apriori_rules = apriori_obj.rules
            
            print("\n✅ Reglas generadas exitosamente. Puedes usar el sistema de recomendaciones.")
        else:
            print("\n⚠️ No se encontraron reglas válidas con la confianza mínima especificada.")
            print("💡 Intenta reducir el valor de confianza mínima.")

# Asignar callbacks
clean_widget_callbacks(btn_calcular_itemsets)
btn_calcular_itemsets.on_click(on_calcular_itemsets)
clean_widget_callbacks(btn_generar_reglas)
btn_generar_reglas.on_click(on_generar_reglas)

# ======================== TAB C: SISTEMA DE RECOMENDACIÓN ========================

# Widgets para recomendación
productos_disponibles = widgets.Select(options=[], description='Productos:', rows=10)
btn_agregar_carrito = widgets.Button(description="Agregar al Carrito", button_style='info')
carrito_items = widgets.SelectMultiple(options=[], description='Mi Carrito:', rows=5)
btn_obtener_recomendaciones = widgets.Button(description="Obtener Recomendaciones", button_style='success')
btn_finalizar_compra = widgets.Button(description="Finalizar Compra", button_style='warning')
output_recomendaciones = widgets.Output()

# Variables globales
carrito_actual = []
apriori_rules = []

def cargar_productos_disponibles():
    """Carga la lista de productos disponibles"""
    productos = list(productos_col.find({}, {"nombre": 1, "_id": 0}))
    productos_nombres = sorted([p['nombre'] for p in productos])
    productos_disponibles.options = productos_nombres

def on_agregar_carrito(b):
    with output_recomendaciones:
        if productos_disponibles.value and productos_disponibles.value not in carrito_actual:
            carrito_actual.append(productos_disponibles.value)
            carrito_items.options = carrito_actual
            print(f"✅ Agregado al carrito: {productos_disponibles.value}")

def obtener_recomendaciones(carrito):
    """Obtiene recomendaciones basadas en las reglas de asociación"""
    if not apriori_rules:
        return []
    
    recomendaciones = []
    carrito_set = set(carrito)
    
    for rule in apriori_rules:
        antecedent_set = set(rule['antecedent'])
        consequent_set = set(rule['consequent'])
        
        # Si el antecedente está contenido en el carrito
        if antecedent_set.issubset(carrito_set):
            # Recomendar items del consecuente que no estén en el carrito
            for item in consequent_set:
                if item not in carrito_set:
                    recomendaciones.append({
                        'producto': item,
                        'confidence': rule['confidence'],
                        'regla': f"{', '.join(rule['antecedent'])} → {', '.join(rule['consequent'])}"
                    })
    
    # Ordenar por confianza y tomar los top 5 (aumentado de 3)
    recomendaciones.sort(key=lambda x: x['confidence'], reverse=True)
    return recomendaciones[:5]

def on_obtener_recomendaciones(b):
    with output_recomendaciones:
        output_recomendaciones.clear_output()
        if not carrito_actual:
            print("🛒 Tu carrito está vacío. Agrega algunos productos primero.")
            return
        
        print("🛒 TU CARRITO ACTUAL:")
        for item in carrito_actual:
            print(f"  • {item}")
        
        recomendaciones = obtener_recomendaciones(carrito_actual)
        
        if recomendaciones:
            print(f"\n💡 RECOMENDACIONES PARA TI:")
            print("=" * 40)
            for i, rec in enumerate(recomendaciones, 1):
                print(f"{i}. {rec['producto']}")
                print(f"   Confianza: {rec['confidence']:.1%}")
                print(f"   Basado en: {rec['regla']}")
                print()
        else:
            print("\n🤷 No se encontraron recomendaciones específicas para tu carrito.")
            if not apriori_rules:
                print("💡 Asegúrate de ejecutar el algoritmo Apriori primero en la pestaña B.")

def on_finalizar_compra(b):
    with output_recomendaciones:
        output_recomendaciones.clear_output()
        if not carrito_actual:
            print("🛒 Tu carrito está vacío.")
            return
        
        # Crear nueva transacción
        nueva_transaccion = {
            '_id': str(uuid.uuid4()),
            'items': carrito_actual.copy(),
            'timestamp': datetime.now(),
            'tipo': 'compra_recomendacion'
        }
        
        # Insertar en MongoDB
        collection.insert_one(nueva_transaccion)
        
        print("🎉 ¡COMPRA FINALIZADA!")
        print("=" * 30)
        print("Productos comprados:")
        for item in carrito_actual:
            print(f"  • {item}")
        
        print(f"\n✅ Transacción guardada con ID: {nueva_transaccion['_id']}")
        
        # Limpiar carrito
        carrito_actual.clear()
        carrito_items.options = []
        
        print("\n🛒 Carrito limpiado. ¡Gracias por tu compra!")

# Asignar callbacks
clean_widget_callbacks(btn_agregar_carrito)
btn_agregar_carrito.on_click(on_agregar_carrito)
clean_widget_callbacks(btn_obtener_recomendaciones)
btn_obtener_recomendaciones.on_click(on_obtener_recomendaciones)
clean_widget_callbacks(btn_finalizar_compra)
btn_finalizar_compra.on_click(on_finalizar_compra)

# Función para actualizar productos disponibles
def actualizar_productos():
    """Actualiza la lista de productos disponibles"""
    cargar_productos_disponibles()

# Cargar productos al inicio
try:
    cargar_productos_disponibles()
except:
    pass  # Si no hay productos aún, se cargarán después de generar el maestro

In [44]:
# ======================== INTERFAZ DE USUARIO ========================

# Crear las pestañas
tab_extraccion = widgets.VBox([
    widgets.HTML("<h2>📊 A. EXTRACCIÓN DE DATOS</h2>"),
    widgets.HTML("<h3>📤 Subir archivo Excel de transacciones</h3>"),
    file_picker,
    ruta_textbox,
    btn_subir,
    progress,
    widgets.HTML("<h3>🛒 Maestro y Análisis Básico</h3>"),
    widgets.HBox([btn_generar_maestro, btn_resumen]),
    output_carga
])

tab_apriori = widgets.VBox([
    widgets.HTML("<h2>🔗 B. ALGORITMO APRIORI Y REGLAS DE ASOCIACIÓN</h2>"),
    
    # ETAPA 1: Itemsets Frecuentes
    widgets.HTML("""
    <div style='background-color: #e3f2fd; padding: 15px; border-radius: 8px; margin: 10px 0;'>
        <h3>📊 ETAPA 1: Cálculo de Itemsets Frecuentes</h3>
        <p>El algoritmo Apriori encuentra combinaciones de productos que aparecen juntos frecuentemente:</p>
        <ul>
            <li><b>1-itemsets:</b> Productos individuales frecuentes</li>
            <li><b>2-itemsets:</b> Pares de productos frecuentes</li>
            <li><b>3-itemsets:</b> Tríos de productos frecuentes</li>
            <li><b>Y así sucesivamente...</b> hasta que no hay más combinaciones frecuentes</li>
        </ul>
    </div>
    """),
    minsup_slider,
    btn_calcular_itemsets,
    output_itemsets,
    
    # ETAPA 2: Reglas de Asociación
    widgets.HTML("""
    <div style='background-color: #f3e5f5; padding: 15px; border-radius: 8px; margin: 20px 0 10px 0;'>
        <h3>🔗 ETAPA 2: Generación de Reglas de Asociación</h3>
        <p>A partir de los itemsets frecuentes, se generan reglas del tipo "Si compra X, entonces comprará Y"</p>
        <p><b>Nota:</b> Primero debes calcular los itemsets frecuentes antes de generar las reglas.</p>
    </div>
    """),
    minconf_slider,
    btn_generar_reglas,
    output_reglas,
    
    # Teoría al final
    widgets.HTML("""
    <div style='background-color: #fffde7; padding: 15px; border-left: 4px solid #f57f17; margin: 20px 0;'>
        <h4>📖 Conceptos Clave:</h4>
        <p><b>Soporte:</b> Porcentaje de transacciones que contienen el itemset<br>
        <b>Confianza:</b> Probabilidad de comprar Y dado que se compró X<br>
        <b>Lift:</b> Cuántas veces más probable es Y cuando se compra X (lift > 1 indica asociación positiva)</p>
        
        <h4>💡 Recomendaciones de uso:</h4>
        <ul>
            <li>Para muchos productos únicos: soporte bajo (1-5%)</li>
            <li>Para pocos productos únicos: soporte medio-alto (10-30%)</li>
            <li>Confianza: generalmente 40-70% para reglas confiables</li>
        </ul>
    </div>
    """)
])

tab_recomendacion = widgets.VBox([
    widgets.HTML("<h2>🛍️ C. SISTEMA DE RECOMENDACIÓN</h2>"),
    widgets.HTML("<p><b>Simula una compra y recibe recomendaciones basadas en las reglas de asociación:</b></p>"),
    widgets.HBox([
        widgets.VBox([
            widgets.HTML("<h4>📦 Productos Disponibles</h4>"),
            productos_disponibles,
            btn_agregar_carrito,
            widgets.HTML("""
            <div style='background-color: #fff3cd; padding: 8px; border-left: 4px solid #ffc107; margin: 5px 0; font-size: 12px;'>
                💡 <b>Tip:</b> Primero ejecuta el Apriori en la pestaña B para generar reglas
            </div>
            """)
        ], layout=widgets.Layout(width='48%')),
        widgets.VBox([
            widgets.HTML("<h4>🛒 Mi Carrito</h4>"),
            carrito_items,
            widgets.HBox([btn_obtener_recomendaciones, btn_finalizar_compra])
        ], layout=widgets.Layout(width='48%'))
    ]),
    output_recomendaciones
])

# Crear el widget de pestañas
tabs = widgets.Tab(children=[tab_extraccion, tab_apriori, tab_recomendacion])
tabs.set_title(0, 'Carga de datos')
tabs.set_title(1, 'Reglas de asociacion')  
tabs.set_title(2, 'Recomendaciones')

# Función auxiliar para actualizar productos cuando se cambie a la pestaña C
def on_tab_change(change):
    if change['new'] == 2:  # Pestaña de recomendaciones
        try:
            actualizar_productos()
        except:
            pass

tabs.observe(on_tab_change, names='selected_index')

# Mostrar la interfaz principal
display(widgets.HTML("""
<div style='text-align: center; background: linear-gradient(90deg, #667eea 0%, #764ba2 100%); 
            color: white; padding: 20px; border-radius: 10px; margin-bottom: 20px;'>
    <h1>🛒 MARKET BASKET ANALYSIS</h1>
    <p>Sistema completo de análisis de cesta de mercado con recomendaciones inteligentes</p>
</div>
"""))

display(tabs)

# Mostrar instrucciones de uso
display(widgets.HTML("""
<div style='background-color: #e8f5e8; padding: 15px; border-radius: 8px; margin-top: 20px;'>
    <h3>📋 INSTRUCCIONES DE USO:</h3>
    <ol>
        <li><b>Pestaña A:</b> Sube tu archivo Excel → Genera el maestro de productos → Ve el resumen</li>
        <li><b>Pestaña B:</b> Ajusta parámetros de soporte y confianza → Ejecuta Apriori para generar reglas</li>
        <li><b>Pestaña C:</b> Agrega productos a tu carrito → Obtén recomendaciones → Finaliza la compra</li>
    </ol>
    <p><b>🔄 Flujo completo:</b> A → B → C</p>
    <p><b>⚙️ Configuración recomendada para datasets con muchos productos:</b><br>
    &nbsp;&nbsp;• Soporte mínimo: 1-5%<br>
    &nbsp;&nbsp;• Confianza mínima: 40-70%</p>
</div>
"""))

HTML(value="\n<div style='text-align: center; background: linear-gradient(90deg, #667eea 0%, #764ba2 100%); \n…

HTML(value="\n<div style='background-color: #e8f5e8; padding: 15px; border-radius: 8px; margin-top: 20px;'>\n …